In [1]:
import os
import time
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image

In [ ]:
def resize_with_padding(img, ratio, new_size, padding_color=255):
    # Open the image
    # Get the new dimensions based on the ratio
    new_width = int(img.width * ratio)
    new_height = int(img.height * ratio)
    
    # Resize the image using the new dimensions
    resized_img = img.resize((new_width, new_height), Image.BILINEAR)
    
    # Create a new blank image with padding
    padded_img = Image.new("L", (new_size, new_size), padding_color)
    
    # Calculate the position to paste the resized image
    left = (padded_img.width - resized_img.width) // 2
    top = (padded_img.height - resized_img.height) // 2
    right = left + resized_img.width
    bottom = top + resized_img.height
    
    # Paste the resized image onto the blank image
    padded_img.paste(resized_img, (left, top, right, bottom))
    
    return padded_img

def get_unpadded_borders(img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    processed = cv2.morphologyEx(np.array(img), cv2.MORPH_CLOSE, kernel)
    npimg = 255 - np.array(processed)
    npimg[np.where(npimg < 16)] = 0
    wmin = npimg.sum(0).nonzero()[0].min()
    wmax = npimg.sum(0).nonzero()[0].max()
    hmin = npimg.sum(1).nonzero()[0].min()
    hmax = npimg.sum(1).nonzero()[0].max()
    return wmin, wmax, hmin, hmax

def augmenting(img):
    img = np.array(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))

    blurred = cv2.GaussianBlur(img, (3, 3), 0)
    processed = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)
    closing = Image.fromarray(processed)
    # err = cv2.erode(processed, kernel)
    # erode = Image.fromarray(err)
    # dil = cv2.dilate(processed, kernel)
    # dilate = Image.fromarray(dil)
    return {
        "closing": closing,
        # "erode": erode,
        # "dilate":dilate,
    }